In [6]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np

In [7]:
df = pd.read_csv("../../assets/survey/2025_sku_sw_survey_data.csv")

In [8]:
# 학년 -> 숫자로 변환
grade = {"1학년" : 1, "2학년" : 2, "3학년" : 3, "4학년" : 4}
df["1) 2025 기준 현재 학년"] = df["1) 2025 기준 현재 학년"].map(grade)

# 목표 직무 -> 학습에서 제외
df.drop(["2) 목표로 하는 직무 (중복선택가능)"], axis = 1, inplace=True)

# 직무 확정 시기 -> TARGET으로 사용
job_time = {"1학년" : 1, "2학년" : 2, "3학년" : 3, "4학년" : 4, "아직 미정" : 0}
df["3) 목표 직무를 확정한 시기"] = df["3) 목표 직무를 확정한 시기"].map(job_time)

# 프로젝트 경험 -> 숫자로 변환
project_cnt = {"없다" : 0, "1번" : 1, "2번" : 2, "3번 이상" : 3}
df["4) 프로젝트 경험 수"] = df["4) 프로젝트 경험 수"].map(project_cnt)

# 전공 난이도 -> 숫자로 변환
difficulty = {"매우 쉽다" : 1, "쉽다" : 2, "보통이다" : 3, "어렵다" : 4, "매우 어렵다" : 5}
df["5) 현재 느끼는 전공 난이도"] = df["5) 현재 느끼는 전공 난이도"].map(difficulty)

# 학과 커리큘럼 만족도 -> 숫자로 변환
satisfaction = {"전혀 아니다" : 1, "아니다" : 2, "보통이다" : 3, "그렇다" : 4, "매우 그렇다" : 5}
df["6) 나는 현재 학과 커리큘럼에 대해 만족한다."] = df["6) 나는 현재 학과 커리큘럼에 대해 만족한다."].map(satisfaction)

# 보유 기술 스택(다중선택) -> 각각의 칼럼으로 변경
df["스택리스트"] = df["7) 현재 보유하고 있는 기술 스택 (중복선택가능)"].fillna("")

def split_input(input): # 스택들 리스트로 저장하는 함수
    if not input:
        return []
    return [s.strip() for s in input.split(",")]

df["스택리스트"] = df["스택리스트"].apply(split_input)

skills = sorted(set(sum(df["스택리스트"], [])))

for skill in skills:
    df[f"skill_{skill}"] = df["스택리스트"].apply(lambda x : 1 if skill in x else 0) # 기술별로 칼럼 다시 생성

df.drop(["7) 현재 보유하고 있는 기술 스택 (중복선택가능)", "스택리스트"], axis=1, inplace=True) # 사용한 칼럼 삭제

# 외부 학습 의존도 -> 숫자로 변환
df["8) 외부 학습 의존도(유튜브, 인프런, 부트캠프, 스터디, 커뮤니티)"] = df["8) 외부 학습 의존도(유튜브, 인프런, 부트캠프, 스터디, 커뮤니티)"].map(satisfaction)

# TARGET 설정
targets = ["3) 목표 직무를 확정한 시기", "10) 나는 원하는 직무에서 일하기 위해 전공 강의만으로는 부족하다고 생각한다.", "11) 나는 전공 강의 이외의 추가 학습이 더 필요하다.", "13) 나는 각 직무별로 취업하기 위해 어떤 것을 공부해야 하는지 명확하게 알고 있다."]

for target in targets[1:]:
    df[target] = df[target].map(satisfaction)

drops = targets + ["타임스탬프", "12) 나는 회사에서 직무별로 요구하는 기술 스택이 상이하다고 생각한다."]
x = df.drop(drops, axis=1) # 입력
y = df[targets].values # 출력


In [10]:
# 정규화
scaler = StandardScaler()
x = scaler.fit_transform(x)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=42) # 일단 고정